In [ ]:
import nannyml as nml
from IPython.display import display

# Load synthetic data
reference = nml.load_synthetic_binary_classification_dataset()[0]
analysis = nml.load_synthetic_binary_classification_dataset()[1]
display(reference.head())

,distance_from_office,salary_range,gas_price_per_litre,public_transportation_cost,wfh_prev_workday,workday,tenure,identifier,work_home_actual,timestamp,y_pred_proba,period,y_pred
0,5.962247,40K - 60K €,2.119485,8.568058,False,Friday,0.212653,0,1,2014-05-09 22:27:20,0.99,reference,1
1,0.535872,40K - 60K €,2.357199,5.425382,True,Tuesday,4.927549,1,0,2014-05-09 22:59:32,0.07,reference,0
2,1.969519,40K - 60K €,2.366849,8.247158,False,Monday,0.520817,2,1,2014-05-09 23:48:25,1.00,reference,1
3,2.530410,20K - 40K €,2.318722,7.944251,False,Tuesday,0.453649,3,1,2014-05-10 01:12:09,0.98,reference,1
4,2.253635,60K+ €,2.221265,8.884478,True,Thursday,5.695263,4,1,2014-05-10 02:21:34,0.99,reference,1


In [ ]:
print(reference.head().to_markdown(tablefmt="grid"))

+----+------------------------+----------------+-----------------------+------------------------------+--------------------+-----------+----------+--------------+--------------------+---------------------+----------------+-----------+----------+
|    |   distance_from_office | salary_range   |   gas_price_per_litre |   public_transportation_cost | wfh_prev_workday   | workday   |   tenure |   identifier |   work_home_actual | timestamp           |   y_pred_proba | period    |   y_pred |
+====+========================+================+=======================+==============================+====================+===========+==========+==============+====================+=====================+================+===========+==========+
|  0 |               5.96225  | 40K - 60K €    |               2.11948 |                      8.56806 | False              | Friday    | 0.212653 |            0 |                  1 | 2014-05-09 22:27:20 |           0.99 | reference |        1 |
+----+----------

In [ ]:
# Define feature columns
feature_column_names = [
    col for col in reference.columns if col not in [
        'timestamp', 'y_pred_proba', 'period', 'y_pred', 'work_home_actual', 'identifier'
    ]]

calc = nml.DataReconstructionDriftCalculator(
    column_names=feature_column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000
)
calc.fit(reference)
results = calc.calculate(analysis)

In [ ]:
display(results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2017-08-31 04:20:00   
1    [5000:9999]           1        5000      9999 2018-01-02 01:13:11   
2  [10000:14999]           2       10000     14999 2018-05-01 14:25:25   
3  [15000:19999]           3       15000     19999 2018-09-01 16:19:07   
4  [20000:24999]           4       20000     24999 2018-12-31 10:38:45   
5  [25000:29999]           5       25000     29999 2019-04-30 11:02:00   
6  [30000:34999]           6       30000     34999 2019-09-01 00:28:54   
7  [35000:39999]           7       35000     39999 2019-12-31 10:07:15   
8  [40000:44999]           8       40000     44999 2020-04-30 12:04:32   
9  [45000:49999]           9       45000     49999 2020-09-01 02:46:13   

                                reconstruction_error            \
             end_date    period       sampling_error     value   
0 2018-01-02 00:45:44  analysis             0.006962  1.118536   
1 2018-05-01 13:10:10  analysis             0.006962  1.115044   
2 2018-09-01 15:40:40  analysis             0.006962  1.125460   
3 2018-12-31 10:11:21  analysis             0.006962  1.128453   
4 2019-04-30 11:01:30  analysis             0.006962  1.122892   
5 2019-09-01 00:24:27  analysis             0.006962  1.228393   
6 2019-12-31 09:09:12  analysis             0.006962  1.220028   
7 2020-04-30 11:46:53  analysis             0.006962  1.237394   
8 2020-09-01 02:46:02  analysis             0.006962  1.206051   
9 2021-01-01 04:29:32  analysis             0.006962  1.242579   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  1.139423                  1.097650        1.138013   
1                  1.135930                  1.094157        1.138013   
2                  1.146346                  1.104573        1.138013   
3                  1.149339                  1.107567        1.138013   
4                  1.143779                  1.102006        1.138013   
5                  1.249280                  1.207507        1.138013   
6                  1.240914                  1.199142        1.138013   
7                  1.258280                  1.216508        1.138013   
8                  1.226938                  1.185165        1.138013   
9                  1.263465                  1.221693        1.138013   

                          
  lower_threshold  alert  
0        1.096577  False  
1        1.096577  False  
2        1.096577  False  
3        1.096577  False  
4        1.096577  False  
5        1.096577   True  
6        1.096577   True  
7        1.096577   True  
8        1.096577   True  
9        1.096577   True

In [ ]:
print(results.filter(period='analysis').to_df().to_markdown(tablefmt="grid"))

+----+--------------------+----------------------------+----------------------------+--------------------------+---------------------------+-------------------------+-----------------------+----------------------------------------------+-------------------------------------+---------------------------------------------------------+---------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+-------------------------------------+
|    | ('chunk', 'key')   |   ('chunk', 'chunk_index') |   ('chunk', 'start_index') |   ('chunk', 'end_index') | ('chunk', 'start_date')   | ('chunk', 'end_date')   | ('chunk', 'period')   |   ('reconstruction_error', 'sampling_error') |   ('reconstruction_error', 'value') |   ('reconstruction_error', 'upper_confidence_boundary') |   ('reconstruction_error', 'lower_confidence_boundary') |   ('reconstruction_error', 'upper_threshold') |   ('reconstruction_error', 'lower_

In [ ]:
display(results.filter(period='reference').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2014-05-09 22:27:20   
1    [5000:9999]           1        5000      9999 2014-09-09 09:13:35   
2  [10000:14999]           2       10000     14999 2015-01-09 00:04:43   
3  [15000:19999]           3       15000     19999 2015-05-09 16:02:08   
4  [20000:24999]           4       20000     24999 2015-09-07 07:27:47   
5  [25000:29999]           5       25000     29999 2016-01-08 17:22:00   
6  [30000:34999]           6       30000     34999 2016-05-09 11:19:36   
7  [35000:39999]           7       35000     39999 2016-09-04 04:09:35   
8  [40000:44999]           8       40000     44999 2017-01-03 19:00:51   
9  [45000:49999]           9       45000     49999 2017-05-03 02:49:38   

                                 reconstruction_error            \
             end_date     period       sampling_error     value   
0 2014-09-09 08:18:27  reference             0.006962  1.120961   
1 2015-01-09 00:02:51  reference             0.006962  1.118071   
2 2015-05-09 15:54:26  reference             0.006962  1.117237   
3 2015-09-07 07:14:37  reference             0.006962  1.125514   
4 2016-01-08 16:02:05  reference             0.006962  1.109446   
5 2016-05-09 11:09:39  reference             0.006962  1.122759   
6 2016-09-04 03:30:35  reference             0.006962  1.107138   
7 2017-01-03 18:48:21  reference             0.006962  1.127134   
8 2017-05-03 02:34:24  reference             0.006962  1.114237   
9 2017-08-31 03:10:29  reference             0.006962  1.110450   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  1.141847                  1.100075        1.138013   
1                  1.138957                  1.097185        1.138013   
2                  1.138123                  1.096351        1.138013   
3                  1.146401                  1.104628        1.138013   
4                  1.130332                  1.088559        1.138013   
5                  1.143645                  1.101872        1.138013   
6                  1.128024                  1.086251        1.138013   
7                  1.148020                  1.106247        1.138013   
8                  1.135123                  1.093350        1.138013   
9                  1.131337                  1.089564        1.138013   

                          
  lower_threshold  alert  
0        1.096577  False  
1        1.096577  False  
2        1.096577  False  
3        1.096577  False  
4        1.096577  False  
5        1.096577  False  
6        1.096577  False  
7        1.096577  False  
8        1.096577  False  
9        1.096577  False

In [ ]:
print(results.filter(period='reference').to_df().to_markdown(tablefmt="grid"))

+----+--------------------+----------------------------+----------------------------+--------------------------+---------------------------+-------------------------+-----------------------+----------------------------------------------+-------------------------------------+---------------------------------------------------------+---------------------------------------------------------+-----------------------------------------------+-----------------------------------------------+-------------------------------------+
|    | ('chunk', 'key')   |   ('chunk', 'chunk_index') |   ('chunk', 'start_index') |   ('chunk', 'end_index') | ('chunk', 'start_date')   | ('chunk', 'end_date')   | ('chunk', 'period')   |   ('reconstruction_error', 'sampling_error') |   ('reconstruction_error', 'value') |   ('reconstruction_error', 'upper_confidence_boundary') |   ('reconstruction_error', 'lower_confidence_boundary') |   ('reconstruction_error', 'upper_threshold') |   ('reconstruction_error', 'lower_

In [ ]:
figure = results.plot()
figure.show()

In [ ]:
figure.write_image('../_static/drift-guide-multivariate.svg')


In [ ]:
# Define feature columns
feature_column_names = [
    col for col in reference.columns if col not in [
        'timestamp', 'y_pred_proba', 'period', 'y_pred', 'work_home_actual', 'identifier'
    ]]

from sklearn.impute import SimpleImputer

calc = nml.DataReconstructionDriftCalculator(
    column_names=feature_column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000,
    imputer_categorical=SimpleImputer(strategy='constant', fill_value='missing'),
    imputer_continuous=SimpleImputer(strategy='median')
)
calc.fit(reference)
results = calc.calculate(analysis)
